In [1]:
# g++ -O3 -shared -o c_functions.so -fPIC c_functions.cpp
import pandas as pd
import numpy as np
import math
import csv
import ctypes
import os
import geopandas as gpd
import time
import concurrent.futures
from rtree import index
from itertools import combinations
from concurrent.futures import ProcessPoolExecutor

In [6]:
# This class holds the information and functions pertaining to each subregion
class Subregion:
    def __init__(self, data):
        self.df = pd.DataFrame(data)
        self.df['featureType'] = self.df['featureType'].astype(str)
        self.featureInfo = {}  # Dictionary to store count, start row ID, and end row ID for each feature type
        self.star_neighbors = {}  # Dictionary to store neighbors of different types within distance_threshold for each instance

    # This function will calculate the feature ranges for each type of feature
    def calc_feature_info(self, offset):
        # Initialize variables to track count, start row ID, and end row ID
        count = 0
        start_row_id = 0
        prev_feature = None
        featureInfo = {}

        # Iterate through the DataFrame
        for i, row in self.df.iterrows():
            feature = row['featureType']

            # If feature type changes, update feature_info for the previous feature
            if feature != prev_feature:
                if prev_feature is not None:
                    featureInfo[prev_feature] = {'count': count, 'start': start_row_id, 'end': i - 1}
                count = 1
                start_row_id = i
                prev_feature = feature
            else:
                count += 1

        # Update feature_info for the last feature
        if prev_feature is not None:
            featureInfo[prev_feature] = {'count': count, 'start': start_row_id, 'end': len(self.df) - 1 + offset}

        # Check if the last feature only has one occurrence
        if prev_feature is not None and count == 1:
            featureInfo[prev_feature] = {'count': count, 'start': start_row_id, 'end': start_row_id}
                
        self.featureInfo = featureInfo
        
    def calc_distance(self, point1, point2):
        x1, y1 = point1['xCoordinate'], point1['yCoordinate']
        x2, y2 = point2['xCoordinate'], point2['yCoordinate']
        return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        
    # new star neighbor function (will be easier to make parallel)
    '''def calc_star_neighbors(self, distance_threshold):
        star_neighbors = {}
        ids = self.df.index.to_numpy()  # indices of df
        coords = np.array(self.df[['xCoordinate', 'yCoordinate']].values)   # x and y values of the original data
        features = np.array(self.df['featureType'])
        unique_features = np.unique(features)
        
        for val_to_exclude in unique_features:
            mask1 = features == val_to_exclude
            mask2 = features != val_to_exclude
            current_feature = coords[mask1]  # coordinates of current feature type
            current_feature_ids = ids[mask1]  # ids of current feature type
            exclude_current_feature = coords[mask2] # coordinates not including the current feature type
            exclude_current_feature_ids = ids[mask2]  # ids not include the current feature type
            
            idx = index.Index()   # create a spatial index
            # insert each point into the spatial index with its unique identifier
            for i, point in enumerate(exclude_current_feature):
                j = exclude_current_feature_ids[i]
                x_coord, y_coord = point[:2]
                idx.insert(j, (x_coord, y_coord, x_coord, y_coord))
                
            # iterate over each point of the current attack type
            for i, point in enumerate(current_feature):
                row_id = current_feature_ids[i]
                x_coord, y_coord = point[:2]
                nearby_points = list(idx.intersection((x_coord - distance_threshold, 
                                                       y_coord - distance_threshold, 
                                                       x_coord + distance_threshold, 
                                                       y_coord + distance_threshold)))
                
                # Filter neighbors based on distance and feature type, and ensure they are greater than the key
                points_to_add = sorted([j for j in nearby_points if j != row_id and 
                                        self.calc_distance(self.df.loc[j], 
                                                           self.df.loc[row_id]) <= distance_threshold and 
                                                           j > row_id])
                star_neighbors[row_id] = points_to_add
            self.star_neighbors = star_neighbors'''
    
    
    def calc_star_neighbors_parallel(self, distance_threshold):
        star_neighbors = {}
        ids = self.df.index.to_numpy()  # Indices of df
        coords = np.array(self.df[['xCoordinate', 'yCoordinate']].values)  # x and y values of the original data
        features = np.array(self.df['featureType'])
        unique_features = np.unique(features)

        # Parallelize over unique features
        with concurrent.futures.ProcessPoolExecutor() as executor:
            futures = [executor.submit(self.process_feature_type, val_to_exclude, features, ids, coords, 
                                       distance_threshold, self.df.copy())  # Copy df for each process
                       for val_to_exclude in unique_features]

            for future in concurrent.futures.as_completed(futures):
                result = future.result()
                star_neighbors.update(result)  # Merge results

        self.star_neighbors = star_neighbors
        
    def process_feature_type(self, val_to_exclude, features, ids, coords, distance_threshold, df):
        star_neighbors = {}
        mask1 = features == val_to_exclude
        mask2 = features != val_to_exclude
        current_feature = coords[mask1]  # Coordinates of current feature type
        current_feature_ids = ids[mask1]  # IDs of current feature type
        exclude_current_feature = coords[mask2]  # Coordinates not including the current feature type
        exclude_current_feature_ids = ids[mask2]  # IDs excluding the current feature type

        idx = index.Index()  # Create a spatial index
        # Insert each point into the spatial index with its unique identifier
        for i, point in enumerate(exclude_current_feature):
            j = exclude_current_feature_ids[i]
            x_coord, y_coord = point[:2]
            idx.insert(j, (x_coord, y_coord, x_coord, y_coord))

        # Iterate over each point of the current feature type
        for i, point in enumerate(current_feature):
            row_id = current_feature_ids[i]
            x_coord, y_coord = point[:2]
            nearby_points = list(idx.intersection((x_coord - distance_threshold, 
                                                   y_coord - distance_threshold, 
                                                   x_coord + distance_threshold, 
                                                   y_coord + distance_threshold)))

            # Filter neighbors based on distance and feature type, and ensure they are greater than the key
            points_to_add = sorted([j for j in nearby_points if j != row_id and 
                                    self.calc_distance(df.loc[j], df.loc[row_id]) <= distance_threshold and 
                                    j > row_id])
            star_neighbors[row_id] = points_to_add

        return star_neighbors
        
def calc_distance(row1, row2):
    # Implement your distance calculation here, assuming it's Euclidean for example
    return np.sqrt((row1['xCoordinate'] - row2['xCoordinate']) ** 2 +
                   (row1['yCoordinate'] - row2['yCoordinate']) ** 2)



In [7]:
# Parameters: These are the only things that need to be changed
# parameters: distance_threshold, prevalance_threshold
with open('distance_threshold_parameter.txt', 'r') as file:   # file to read distance threshold path
    distance_threshold = float(file.read())
distance_threshold =     71.22
prevalence_threshold = 0.55    # set the prevalence threshold
# shapefile path to determine border points
shapefile_path = '/home/amk7r/colocation_mining/compression_code/data/north_america/shapefile'
directory_path = '/home/amk7r/colocation_mining/compression_code/data/north_america'
subregions = []
dataframes = []
offsets = [0]
number_subregions = 0
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)
        df.index = range(offsets[number_subregions], offsets[number_subregions] + len(df))  # apply the offset
        offsets.append(df.shape[number_subregions])
        subregions.append(Subregion(df))
        dataframes.append(df)
        number_subregions += 1
        
offsets.pop()
print(offsets)

[0, 2913]


In [8]:
def process_subregion(i, subregion, offset, distance_threshold):
    subregion.calc_feature_info(offset)
    subregion.calc_star_neighbors_parallel(distance_threshold)
    subregion.features = subregion.featureInfo.keys()
    
    feature_info = [(feature, values['count'], values['start'], values['end'])
                    for feature, values in subregion.featureInfo.items()]
    
    star_neighbors = [(feature, ' '.join(map(str, neighbors)))
                      for feature, neighbors in subregion.star_neighbors.items()]

    return i, feature_info, star_neighbors

def parallel_process_subregions(subregions, offsets, distance_threshold):
    results = []
    with ProcessPoolExecutor() as executor:
        futures = [
            executor.submit(process_subregion, i, subregions[i], offsets[i], distance_threshold)
            for i in range(len(subregions))
        ]
        for future in concurrent.futures.as_completed(futures):
            results.append(future.result())

    # Sequential file I/O after parallel processing
    for i, feature_info, star_neighbors in results:
        with open(f'featureInfoParallel/featureInfo{i}.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['feature', 'count', 'start', 'end'])
            writer.writerows(feature_info)

        sorted_star_neighbors = sorted(star_neighbors, key=lambda x: x[0])
        with open(f'starNeighborsParallel/starNeighbors{i}.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['feature', 'star neighbors'])
            writer.writerows(sorted_star_neighbors)

In [9]:
s = time.time()
# Call the parallel function
parallel_process_subregions(subregions, offsets, distance_threshold)
e = time.time()
print(e - s)

17.20045232772827


original time: 47.42070484161377
parallel time: 17.343128442764282

In [ ]:
lib = ctypes.CDLL('./c_functions.so')

In [6]:
lib.subregion_main(ctypes.c_int(number_subregions), ctypes.c_double(prevalence_threshold))

Degree 2 Prevalent Patterns for Sub-Region 0:
(0, 1)
(0, 2)
(0, 3)
(0, 7)
(1, 2)
(1, 4)
(1, 7)
(2, 3)
(2, 5)
(2, 7)
(3, 7)
Degree 3 Prevalent Patterns for Sub-Region:0:
(0, 1, 2)
(0, 2, 3)
(1, 2, 7)
Degree 2 Prevalent Patterns for Sub-Region 1:
(0, 1)
(0, 2)
(0, 6)
(1, 6)
(2, 6)
(2, 8)
(3, 5)
-0.984562873840332
Degree 3 Prevalent Patterns for Sub-Region:1:


In [7]:
# This class holds the information pertaining to the border region
class Border:
    def __init__(self):
        self.combined_df = pd.DataFrame()
        self.border_df = pd.DataFrame()
        self.featureInfo = {}
        self.star_neighbors = {}
        
    # This function will calculate the feature ranges for each type of feature
    def calc_feature_info(self):
        # Initialize variables to track count, start row ID, and end row ID
        count = 0
        start_row_id = 0
        prev_feature = None
        featureInfo = {}

        # Iterate through the DataFrame
        for i, row in self.border_df.iterrows():
            feature = row['featureType']

            # If feature type changes, update feature_info for the previous feature
            if feature != prev_feature:
                if prev_feature is not None:
                    featureInfo[prev_feature] = {'count': count, 'start': start_row_id, 'end': i - 1}
                count = 1
                start_row_id = i
                prev_feature = feature
            else:
                count += 1

        # Update feature_info for the last feature
        if prev_feature is not None:
            featureInfo[prev_feature] = {'count': count, 'start': start_row_id, 'end': len(self.border_df) - 1}

        # Check if the last feature only has one occurrence
        if prev_feature is not None and count == 1:
            featureInfo[prev_feature] = {'count': count, 'start': start_row_id, 'end': start_row_id}
                
        self.featureInfo = featureInfo
        
    def calc_distance(self, point1, point2):
        x1, y1 = point1['xCoordinate'], point1['yCoordinate']
        x2, y2 = point2['xCoordinate'], point2['yCoordinate']
        return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        
    # This function will calculate the star neighbors for each instance
    def calc_star_neighbors(self, distance_threshold):
        star_neighbors = {}
        # Create a spatial index
        idx = index.Index()
        # Insert each point into the spatial index with its unique identifier
        for i, row in self.border_df.iterrows():
            x_coord = row['xCoordinate']
            y_coord = row['yCoordinate']
            idx.insert(i, (x_coord, y_coord, x_coord, y_coord))

        # Iterate over each row in the DataFrame
        for i, row in self.border_df.iterrows():
            row_id = i
            feature_type = row['featureType']
            x_coord = row['xCoordinate']
            y_coord = row['yCoordinate']

            # Query the spatial index to find nearby points within the distance threshold
            nearby_points = list(idx.intersection((x_coord - distance_threshold, y_coord - distance_threshold, x_coord + distance_threshold, y_coord + distance_threshold)))

            # Filter neighbors based on distance and feature type, and ensure they are greater than the key
            points_to_add = sorted([j for j in nearby_points if j != row_id and self.calc_distance(self.border_df.loc[j], self.border_df.loc[row_id]) <= distance_threshold and 
                                    self.border_df.loc[j, 'featureType'] != feature_type and j > row_id])
            # Store the nearby points in the dictionary
            star_neighbors[row_id] = points_to_add
            
        self.star_neighbors = star_neighbors

In [8]:
border = Border()
border.combined_df = pd.concat(dataframes, ignore_index=True)
border.combined_df['ID'] = border.combined_df.index

# sort the df by featureType
border.combined_df = border.combined_df.sort_values(by='featureType', ignore_index=True)

In [9]:
border.combined_df

,featureType,latitude,longitude,xCoordinate,yCoordinate,ID
0,0,34.607389,-86.979767,-9671.708811,3848.166081,0
1,0,42.880273,-87.900801,-9774.123119,4768.068811,373
2,0,47.462201,-122.254239,-13594.051140,5277.555959,372
3,0,47.242603,-122.454338,-13616.301130,5253.137775,371
4,0,47.658780,-117.426047,-13057.180680,5299.414546,370
...,...,...,...,...,...,...
3547,8,25.774591,-80.214195,-8919.411529,2866.003756,2908
3548,8,25.774591,-80.214195,-8919.411529,2866.003756,2909
3549,8,27.946530,-82.459269,-9169.052368,3107.512353,2910
3550,8,14.540831,-92.220917,-10254.498100,1616.866636,3536


In [10]:
shapefile = gpd.read_file(shapefile_path)
points = np.array(border.combined_df[['latitude', 'longitude']])
points_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy(points[:, 1], points[:, 0]))
points_gdf = points_gdf.set_crs("EPSG:4326")
shapefile = shapefile.to_crs("EPSG:4326")

In [11]:
featureType = []
x = []
y = []
ID = []
curr_index = 0
for point in points_gdf.geometry:
    # Convert radius from kilometers to degrees (assuming a spherical Earth)
    radius_deg = distance_threshold / 111.32  # Approximately 111.32 km per degree of latitude
    # Create a circle geometry
    circle = point.buffer(radius_deg)
    
    # Find the borders that the point intersects
    intersected_borders = [border for border in shapefile['geometry'] if circle.intersects(border)]
    
    if len(intersected_borders) >= 2:
        featureType.append(border.combined_df['featureType'].iloc[curr_index])
        x.append(border.combined_df['xCoordinate'].iloc[curr_index])
        y.append(border.combined_df['yCoordinate'].iloc[curr_index])
        ID.append(border.combined_df['ID'].iloc[curr_index])
    curr_index += 1

In [12]:
border.border_df = pd.DataFrame({
    'featureType': featureType,
    'xCoordinate': x,
    'yCoordinate': y,
    'ID': ID})

In [13]:
border.border_df

,featureType,xCoordinate,yCoordinate,ID
0,0,-13004.57470,3613.132698,2918
1,0,-13004.57470,3613.132698,2917
2,0,-13004.57470,3613.132698,2916
3,0,-13004.57503,3613.132809,2915
4,0,-13004.57503,3613.132809,2914
...,...,...,...,...
85,6,-10842.13160,2874.623141,3517
86,6,-13004.57470,3613.132698,3423
87,6,-12180.44382,3482.860836,3510
88,6,-13004.57470,3613.132698,3424


In [14]:
ids = border.border_df['ID'].to_list()

In [15]:
border.calc_feature_info()
# write the featureInfo to a csv file
with open('border_featureInfo/featureInfo.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['feature', 'count', 'start', 'end'])
    for feature, values in border.featureInfo.items():
        writer.writerow([feature, values['count'], values['start'], values['end']])

In [16]:
border.calc_star_neighbors(distance_threshold)
# write the star_neighbor to a csv file
with open('border_starNeighbors/starNeighbors.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['feature', 'star neighbors'])
    for feature, neighbors in border.star_neighbors.items():
        values_str = ' '.join(map(str, neighbors))
        writer.writerow([feature, values_str])

In [17]:
number_borders = 1
lib.border_main(ctypes.c_int(number_borders), ctypes.c_double(prevalence_threshold))

Degree 2 Prevalent Patterns for Border 0:
(0, 1)
(0, 2)
(0, 3)
(0, 6)
(1, 3)
(1, 6)
(2, 3)
(2, 6)
(3, 6)


0

In [18]:
arr_len = len(ids)
arr_type = ctypes.c_int * arr_len
arr_c = arr_type(*ids)
border_number = 0
lib.update_border_info.argtypes = (ctypes.POINTER(ctypes.c_int), ctypes.c_int, ctypes.c_int)
lib.update_border_info(arr_c, arr_len, border_number)

0

In [19]:
lib.combine_hashmaps.argtypes = (ctypes.c_int, ctypes.c_int)
lib.combine_hashmaps(number_subregions, number_borders)

0

In [20]:
lib.combine_instance_tables.argtypes = (ctypes.c_int, ctypes.c_int)
lib.combine_instance_tables(number_subregions, number_borders)

0

In [ ]:
features = list(border.combined_df['featureType'].unique())
string_ptrs = (ctypes.c_char_p * len(features))()
string_ptrs[:] = [s.encode() for s in features]
lib.region_main.argtypes = (ctypes.c_int, ctypes.c_double, ctypes.POINTER(ctypes.c_char_p), ctypes.c_int)
lib.region_main(number_subregions, prevalence_threshold, string_ptrs, len(features))

Degree 2 Prevalent Patterns for Entire Region:
(0, 1)
(0, 2)
(0, 3)
(1, 2)
(1, 6)
(2, 3)
(2, 5)
(2, 7)
